# LC4RI入門

所要時間：30分

NIIクラウド運用チームの実践する「LC4RI（Literate Computing for Reproducible Infrastructure）」について、お話します。

## Literate Computing とは何か

[コンピューティング](https://ja.wikipedia.org/wiki/%E3%82%B3%E3%83%B3%E3%83%94%E3%83%A5%E3%83%BC%E3%83%86%E3%82%A3%E3%83%B3%E3%82%B0)

["Literate computing" and computational reproducibility: IPython in the age of data-driven journalism](http://blog.fperez.org/2013/04/literate-computing-and-computational.html)

[Notebook interface](https://en.wikipedia.org/wiki/Notebook_interface)

コンピュータとインタラクションしながら作業する時、個々の操作と結果の対の列が文脈を構成します。操作だけの列や、一部が欠けた状態では、文脈が保存されているとは言えません。Pythonの対話型シェル「IPython」の作者である Fernando Perez は、IPythonによるコンピューティングの文脈を保存するために Notebook interface を採用した「IPython Notebook（現 Jupyter Notebook）」を作りました。

コンピュータの操作の列では、その操作の意図が明確に読み取れない、読み取りにくい場合もあるため、ソースコードに含まれるコメントのように、自然言語による注意書きが効果的です。この注意書きを実現するため、Jupyter Notebook には、マークダウンでドキュメンテーションする機能があります。

実際の操作履歴に加え、操作時の背景をメモしておくことで、文脈が保存され、あとから読んだり、再利用したりできるようになります。

## 情報システムの構築・運用

情報システムの構築・運用では、エンジニアがCLIを操作して作業することが多くあります。また、その作業証跡が必要となったり、背景知識を理解してシニアエンジニアと同じ作業をできるようにトレーニングしたり、完全に同じではないが似ている作業を頻繁に行ったりします。情報システムの構築・運用におけるこの特徴が、Literate Computing とマッチすることから、Jupyter Notebook を用いて情報システムの構築・運用を行うアイデアが生まれました。その実践により生まれた手法が「LC4RI」です。

また、「[Infrastructure as Code](https://ja.wikipedia.org/wiki/Infrastructure_as_Code)」の流行によって、情報システムの構築・運用に利用できるツール群が整備されたことも、「LC4RI」の取り組みを進めやすくしました。例えば、「[Ansible](https://www.ansible.com/)」です。 

### 実例：Ansible経由のOSパッケージのアップグレード

Jupyter Notebook が動いている
まずは、ディストリビューションの確認から。

In [ ]:
!cat /etc/issue

Ubuntuなので、 apt-get upgrade してみましょう。

----

まず、操作対象をインベントリファイルに書いておきます。今回の対象は、localhost、1台とします。

In [ ]:
!echo "localhost" > hosts

https://docs.ansible.com/ansible/latest/modules/apt_module.html

aptモジュールを使ってupgradeを指示します。

今回は、対象がローカルなので「-c local」オプションを指定します。これを付けないと、aptモジュールなどLinuxを対象とした操作はssh経由になってしまいます。

今回は、aptの実行に管理者権限が必要なので、「--become」オプションを指定します。

今回は、update後にupgradeするように「-a "update_cache=yes upgrade=yes"」を指定します。

In [ ]:
!ansible -i ./hosts all --become -c local -m apt -a "update_cache=yes upgrade=yes"

（警告はいっぱい出たけど）upgradeに成功した！

----

みたいなNotebookを使った作業を行います。

### 完全に同じではないが似ている作業

1.2.1の手順が Notebook として存在している時、操作対象を「localhost、1台」から「192.168.1.11〜192.168.1.19、9台」に変えて apt-get upgrade したい場合、 hosts の内容を9台の指定とし、「-c local」オプションを省いて ansible コマンドを実行すれば良いでしょう。これを、 Notebook のコピーと改変で行うことができます。

### 背景知識を理解してシニアエンジニアと同じ作業をできる

シニアエンジニアが、ターミナルを使って、さまざまなコマンドを駆使して作業を行った時、その作業内容を他のメンバと共有しないと、知見が属人化します。技能移転も行えません。

シニアエンジニアが、 Notebook を使って、さまざまなコマンドを駆使したり、途中でググったメモを残すなどして作業を行った時、他のメンバと Notebook を共有することで、知見を共有できます。 Notebook を通じて、シニアエンジニアの作業を追体験でき、技能移転が行えます。

### 作業証跡

In [ ]:
!ls -la

In [ ]:
!ls -la .log

今日の日付のディレクトリを覗いてみましょう。

In [ ]:
!ls -la .log/2019xxxx

logファイルを1つ覗いてみましょう。

In [ ]:
!cat .log/2019xxxx/2019xxxx-xxxxxx-xxxx.log

全てのセルの実行結果を、個々にログファイルに残しています。（プラグイン拡張）

ログファイルを生成時に転送する仕組みを用意することで、監査などの目的に使える、作業証跡を残すことができます。

### 手順のメンテナンス

対象システムがアップデートされると、以前の手順が使えなくなる場合があります。このため、以前の手順をコピーし改変して再利用する際には、同様の Notebook の中から、最も直近で動いた実績があるものをコピーするようにします。対象システムの振る舞いが変化し、コピーした手順をそのまま使えない場合、その場で調査などして、 Notebook に修正を施し（当然、その経緯も書き残し）、作業を完了させます。次回には、修正された Notebook がコピー元に使われるため、手順の再利用が行える可能性が高くなります。

マスターを管理し、メンテナンスすることも可能ですが、上記のプロセスよりもコストが掛かります。また、マスターは清書した結果を書きがちなので、文脈が消えてしまい、情報を伝える効果が減ってしまう可能性があります。

### 手順がない作業

初めて発生した障害の復旧など、手順が存在しない作業を行う場合があります。この場合も、新規作成した Notebook をターミナルの代わりに使って作業を行います。すると、冗長でメモが少ないかも知れませんが、新しく有用な手順が手に入ります。シニアが作業した Notebook をジュニアの人が流用して作業できたり、メモが分かりにくいければ質問して議論して追記したり、不要な部分を削除して清書したり、想定外だった作業を手順化し、チームで共有することができます。

### Ansibleについての注意

Ansibleを使うことで、

- Linuxサーバの他にもネットワーク機器を操作することもできます
- インベントリを適切に構成することで、複数の操作対象にまとめて操作を行うことができます
- 機器台帳からインベントリに切り出してくるフローを考えることもできます
- 定番的に使われる手順をplaybook化しても良いかも知れません（ほんとうに適したケースは稀ですが）

といった利点がありますが、チームによっては、Ansibleを使いこなすのが難しいというケースもあるでしょう。そうした場合には、

[[nornir] Python 製自動化フレームワーク「nornir」かんたんチュートリアル（Ansibleと比較しながら）](https://tekunabe.hatenablog.jp/entry/2018/10/16/nornir_intro)

など、他のツールをいろいろ試して、自分たちに合うものを選定すると良いでしょう。

おしまい。